In [1]:
from src.get_data import get_coinmetrics_data

In [1]:
from src.get_data import main
main()

        symbol   status baseAsset  baseAssetPrecision quoteAsset  \
0       ETHBTC  TRADING       ETH                   8        BTC   
1       LTCBTC  TRADING       LTC                   8        BTC   
2       BNBBTC  TRADING       BNB                   8        BTC   
3       NEOBTC  TRADING       NEO                   8        BTC   
4      QTUMETH  TRADING      QTUM                   8        ETH   
...        ...      ...       ...                 ...        ...   
2379   LUNCTRY  TRADING      LUNC                   8        TRY   
2380  SUPERTRY  TRADING     SUPER                   8        TRY   
2381   JTOUSDT  TRADING       JTO                   8       USDT   
2382  JTOFDUSD  TRADING       JTO                   8      FDUSD   
2383    JTOTRY  TRADING       JTO                   8        TRY   

      quotePrecision  quoteAssetPrecision  baseCommissionPrecision  \
0                  8                    8                        8   
1                  8                    8  

KeyError: 'symbols'

In [2]:
data = get_coinmetrics_data(days_before_today=3000)

2023-12-10 21:13:47 INFO     API key not found. Using an empty string.
2023-12-10 21:14:01 INFO     Sleeping for a rate limit window because 429 (too many requests) error was returned. Pleasesee Coin Metrics APIV4 documentation for more information: https://docs.coinmetrics.io/api/v4/#tag/Rate-limits
2023-12-10 21:14:16 INFO     Sleeping for a rate limit window because 429 (too many requests) error was returned. Pleasesee Coin Metrics APIV4 documentation for more information: https://docs.coinmetrics.io/api/v4/#tag/Rate-limits
2023-12-10 21:14:32 INFO     Sleeping for a rate limit window because 429 (too many requests) error was returned. Pleasesee Coin Metrics APIV4 documentation for more information: https://docs.coinmetrics.io/api/v4/#tag/Rate-limits
2023-12-10 21:14:52 INFO     Sleeping for a rate limit window because 429 (too many requests) error was returned. Pleasesee Coin Metrics APIV4 documentation for more information: https://docs.coinmetrics.io/api/v4/#tag/Rate-limits
2023-

In [2]:
main()

NameError: name 'main' is not defined

In [4]:
data.to_parquet('coinmrkets_data.parquet')

In [3]:
from os import environ
import sys
import pandas as pd
import numpy as np
import logging
from datetime import date, datetime, timedelta
from coinmetrics.api_client import CoinMetricsClient
import json
import logging
from typing import List, Dict
from os import environ
import pandas as pd
from coinmetrics.api_client import CoinMetricsClient
from datetime import datetime, timedelta
from typing import List
import pandas as pd
from typing import List
import pandas as pd
from binance.client import Client
import datetime

def get_metrics_names() -> List[str]:
    """
    Read metric names from a file and return them as a list.

    Returns:
    List[str]: A list containing metric names.

    Raises:
    FileNotFoundError: If the specified file is not found.
    Exception: If an error occurs while reading the file.
    """
    # Specify the file path
    file_path = 'input_data/metrics.txt'

    # Initialize an empty list to store metric names
    metric_names = []

    try:
        # Open the file in read mode
        with open(file_path, 'r') as file:
            # Read all lines from the file
            lines = file.readlines()

            # Remove leading and trailing whitespaces and add each line to the list
            metric_names = [line.strip() for line in lines]

    except FileNotFoundError:
        # Handle file not found error
        print(f"File '{file_path}' not found.")
    except Exception as e:
        # Handle other exceptions
        print(f"An error occurred: {e}")

    # Return the list of metric names
    return metric_names


def get_asset_names()-> List[str]:
    """
    Read metric names from a file and return them as a list.

    Returns:
    List[str]: A list containing asset names.

    Raises:
    FileNotFoundError: If the specified file is not found.
    Exception: If an error occurs while reading the file.
    """
    # Specify the file path
    file_path = 'input_data/assets.txt'

     # Initialize an empty list to store asset names
    asset_names = []

    try:
        # Open the file in read mode
        with open(file_path, 'r') as file:
            # Read all lines from the file
            lines = file.readlines()

            # Remove leading and trailing whitespaces and add each line to the list
            asset_names = [line.strip() for line in lines]

    except FileNotFoundError:
        # Handle file not found error
        print(f"File '{file_path}' not found.")
    except Exception as e:
        # Handle other exceptions
        print(f"An error occurred: {e}")

    # Return the list of asset names
    return asset_names




def configure_logger() -> None:
    """
    Configure the logging format and level.

    This function configures the logging format, level, and date format.

    Returns:
    None
    """
    logging.basicConfig(
        format='%(asctime)s %(levelname)-8s %(message)s',
        level=logging.INFO,
        datefmt='%Y-%m-%d %H:%M:%S'
    )

def initialize_coin_metrics_client(api_key: str = "") -> CoinMetricsClient:
    """
    Initialize the CoinMetricsClient with the provided API key.

    This function initializes the CoinMetricsClient using the provided API key.
    If no API key is provided, it uses an empty string.

    Args:
    api_key (str): The API key for accessing CoinMetrics services.

    Returns:
    CoinMetricsClient: An instance of the CoinMetricsClient.
    """
    # Use the API key from the environment if available
    try:
        api_key = environ["CM_API_KEY"]
        logging.info("Using API key found in environment")
    except KeyError:
        logging.info("API key not found. Using an empty string.")

    # Initialize the CoinMetricsClient
    client = CoinMetricsClient(api_key)

    return client

def fetch_asset_metrics(
    client: CoinMetricsClient,
    assets: List[str],
    metrics: List[str],
    start_time: str,
    frequency: str = '1d'
) -> pd.DataFrame:
    """
    Fetch asset metrics using the CoinMetricsClient.

    This function fetches asset metrics using the CoinMetricsClient for specified
    assets, metrics, start time, and frequency.

    Args:
    client (CoinMetricsClient): An instance of the CoinMetricsClient.
    assets (List[str]): A list of asset symbols.
    metrics (List[str]): A list of metric names.
    start_time (str): The start time for fetching metrics data.
    frequency (str): The frequency of the data (default is '1d').

    Returns:
    pd.DataFrame: A pandas DataFrame containing the fetched asset metrics.
    """
    # Fetch asset metrics
    metrics_data = client.get_asset_metrics(
        assets=assets,
        start_time=start_time,
        metrics=metrics,
        frequency=frequency
    ).to_dataframe()

    return metrics_data

In [4]:
configure_logger()
coin_metrics_client = initialize_coin_metrics_client()

# Retrieve asset names and metrics
assets = ['eth']
metrics = ['CapMVRVCur']

# Calculate the start time (3 days before today)
start_time = (datetime.datetime.now() - timedelta(days=3)).strftime('%Y-%m-%d')

# Set the data fetching frequency
frequency = '1d'

# Fetch asset metrics data
metrics_data = fetch_asset_metrics(coin_metrics_client, assets, metrics, start_time, frequency)

2023-12-10 21:08:58 INFO     API key not found. Using an empty string.


In [5]:
metrics_data

,asset,time,CapMVRVCur
0,eth,2023-12-07 00:00:00+00:00,1.838653
1,eth,2023-12-08 00:00:00+00:00,1.839359
2,eth,2023-12-09 00:00:00+00:00,1.827039


In [10]:
from os import environ
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import logging
from datetime import date, datetime, timedelta
from coinmetrics.api_client import CoinMetricsClient
import json
import logging
from pytz import timezone as timezone_conv
from datetime import timezone as timezone_info

import matplotlib.pyplot as plt
%matplotlib inline
sns.set_theme()
sns.set(rc={'figure.figsize':(12,8)})
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)
# We recommend privately storing your API key in you

try:
    api_key = environ["CM_API_KEY"]
    logging.info("Using API key found in environment")
except KeyError:
    api_key = ""
    logging.info("API key not found. Using community client")


client = CoinMetricsClient(api_key)
assets = ['btc']#, 'eth']
asset_mapping = {i: assets[i] for i in range(len(assets))}
print(asset_mapping)

2023-12-11 18:11:19 INFO     API key not found. Using community client


{0: 'btc'}


In [11]:
asset_catalog = client.catalog_assets(assets=assets)
full_asset_catalog = client.catalog_full_assets(assets=assets)

In [12]:
asset_catalog[0]

{'asset': 'btc',
 'full_name': 'Bitcoin',
 'metrics': [{'metric': 'AdrActCnt',
   'frequencies': [{'frequency': '1d',
     'min_time': '2009-01-03T00:00:00.000000000Z',
     'max_time': '2023-12-10T00:00:00.000000000Z',
     'community': True}]},
  {'metric': 'AdrBal1in100KCnt',
   'frequencies': [{'frequency': '1d',
     'min_time': '2009-01-03T00:00:00.000000000Z',
     'max_time': '2023-12-10T00:00:00.000000000Z',
     'community': True}]},
  {'metric': 'AdrBal1in100MCnt',
   'frequencies': [{'frequency': '1d',
     'min_time': '2009-01-03T00:00:00.000000000Z',
     'max_time': '2023-12-10T00:00:00.000000000Z',
     'community': True}]},
  {'metric': 'AdrBal1in10BCnt',
   'frequencies': [{'frequency': '1d',
     'min_time': '2009-01-03T00:00:00.000000000Z',
     'max_time': '2023-12-10T00:00:00.000000000Z',
     'community': True}]},
  {'metric': 'AdrBal1in10KCnt',
   'frequencies': [{'frequency': '1d',
     'min_time': '2009-01-03T00:00:00.000000000Z',
     'max_time': '2023-12-10T

In [36]:
j = []
for i in range(1500):
    temps = pd.DataFrame.from_dict(full_asset_catalog[0]['metrics'][i]['frequencies'])
    if temps['frequency'].to_list()[0] == '1d':
        print(temps)
        j.append(i)


  frequency                        min_time                        max_time
0        1d  2009-01-03T00:00:00.000000000Z  2023-12-10T00:00:00.000000000Z
  frequency                        min_time                        max_time
0        1d  2009-01-03T00:00:00.000000000Z  2023-12-10T00:00:00.000000000Z
  frequency                        min_time                        max_time  \
0        1d  2009-01-03T00:00:00.000000000Z  2023-12-10T00:00:00.000000000Z   

   community  
0       True  
  frequency                        min_time                        max_time  \
0        1d  2009-01-03T00:00:00.000000000Z  2023-12-10T00:00:00.000000000Z   

   community  
0       True  
  frequency                        min_time                        max_time  \
0        1d  2009-01-03T00:00:00.000000000Z  2023-12-10T00:00:00.000000000Z   

   community  
0       True  
  frequency                        min_time                        max_time  \
0        1d  2009-01-03T00:00:00.000000000Z  2023-

IndexError: list index out of range

In [37]:
len(j)

356

In [33]:
pd.DataFrame.from_dict(full_asset_catalog[0]['metrics'])['metric']#.to_list()

0                       AdrAct30dCnt
1                        AdrAct7dCnt
2                          AdrActCnt
3                       AdrActRecCnt
4                      AdrActSentCnt
                   ...              
507    volume_reported_future_usd_1h
508      volume_reported_spot_usd_1d
509      volume_reported_spot_usd_1h
510       volume_trusted_spot_usd_1d
511       volume_trusted_spot_usd_1h
Name: metric, Length: 512, dtype: object

In [34]:
pd.DataFrame.from_dict(full_asset_catalog[0]['metrics'])['metric'].to_list()

['AdrAct30dCnt',
 'AdrAct7dCnt',
 'AdrActCnt',
 'AdrActRecCnt',
 'AdrActSentCnt',
 'AdrBal1in100KCnt',
 'AdrBal1in100MCnt',
 'AdrBal1in10BCnt',
 'AdrBal1in10KCnt',
 'AdrBal1in10MCnt',
 'AdrBal1in1BCnt',
 'AdrBal1in1KCnt',
 'AdrBal1in1MCnt',
 'AdrBalCnt',
 'AdrBalNtv0.001Cnt',
 'AdrBalNtv0.01Cnt',
 'AdrBalNtv0.1Cnt',
 'AdrBalNtv100Cnt',
 'AdrBalNtv100KCnt',
 'AdrBalNtv10Cnt',
 'AdrBalNtv10KCnt',
 'AdrBalNtv1Cnt',
 'AdrBalNtv1KCnt',
 'AdrBalNtv1MCnt',
 'AdrBalUSD100Cnt',
 'AdrBalUSD100KCnt',
 'AdrBalUSD10Cnt',
 'AdrBalUSD10KCnt',
 'AdrBalUSD10MCnt',
 'AdrBalUSD1Cnt',
 'AdrBalUSD1KCnt',
 'AdrBalUSD1MCnt',
 'AdrNewBalCnt',
 'AdrNewCnt',
 'AssetEODCompletionTime',
 'BlkCnt',
 'BlkHgt',
 'BlkIntMean',
 'BlkSizeByte',
 'BlkSizeMeanByte',
 'BlkWghtMean',
 'BlkWghtTot',
 'CapAct1yrUSD',
 'CapFutExp10yrUSD',
 'CapMVRVCur',
 'CapMVRVFF',
 'CapMrktCurUSD',
 'CapMrktEstUSD',
 'CapMrktFFUSD',
 'CapRealUSD',
 'DiffLast',
 'DiffMean',
 'FeeByteMeanNtv',
 'FeeMeanNtv',
 'FeeMeanUSD',
 'FeeMedNtv',
 'Fe